# DG Basics: Projection and Approximation Properties 

## What's new

- Creating numerical grids
- Projecting functions onto the DG space and further evaluation
- Performing a hp-convergence study


### Prerequisites

No **BoSSS** specific prerequisites are needed to complete this tutorial.


## Problem statement
>First, we define two functions:
> $g_1$ is continuous, $g_2$ has a discontinuity at $x = \pi$ in the first derivative

>$$ g_1(x) := sin(x), $$
>$$g_2(x) := \vert sin(x) \vert ,$$

The function argument is a vector $x \in R^n$, consisting only of one entry 
since we are working in a one dimensional space. 

**BoSSS** however supports 1D, 2D and 3D, so the spatial coordinate 
is a general vector.



## Solution within the BoSSS framework

Note: 
1. This tutorial can be found in the source code repository as asd `ue2Basics.ipynb`. 
   One can directly load this into Jupyter to interactively work with the following code examples.
2. **In the following line, the reference to `BoSSSpad.dll` is required**. 
   You must either set `#r "BoSSSpad.dll"` to something which is appropirate for your computer
   (e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if you installed the binary distribution),
   or, if you are working with the source code, you must compile `BoSSSpad` and put it side-by-side to this worksheet file
   (from the original location in the repository, you can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).

In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
using NUnit.Framework;

### Plotting the functions

 First, we plot the functions that are defined above over the interval $(0 ,2 \pi)$ with 
1000 sampling points using a Gnuplot-object.

In [3]:
Func<double[],double> g1 = (X => Math.Sin(X[0]));          
// continuous, smooth         
Func<double[],double> g2 = (X => Math.Abs(Math.Sin(X[0])));          
// continuous, non-smooth

We define equidistant sampling points...

In [4]:

double[] x = GenericBlas.Linspace(0, 2.0*Math.PI, 1000);

...and compute the function values. In the loop, we have to convert the scalar **x[i]** into an array
with one element, since $g_1$ has to be feed with arrays.

In [5]:

double[] g1_values = new double[x.Length];
for(int i = 0; i < x.Length; i++) {
    g1_values[i] = g1(new[] { x[i]} );
}

In [6]:
/// Instead of loops, we can also use Linq-functions:
double[] g2_values = x.Select(x => g2(new []{ x })).ToArray();

For now, we are using the simple plotting interface, which supports
Matlab-Style format specifiers and color names. 

(More advanced plots can be produced with **Plot2Ddata** and/or **Gnuplot** classes)

>## Note
>In order to obtain an output for the plot or any other command, there must not be a semicolon **;** 
> at the end of the line! 

In [7]:
Plot(X1:x, Y1:g1_values, Name1:"function g1", Format1:"--red",
     X2:x, Y2:g2_values, Name2:"function g2", Format2:"-.blue")

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 -0.8 
 
 
 
 
 -0.6 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 function g1 
 
 
 function g1 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='2.5,4.0' d='M940.2,34.7 L982.4,34.7 M53.9,374.3 L54.7,372.1 L55.6,369.9 L56.4,367.6 L57.3,365.4 L58.1,363.1
 L59.0,360.9 L59.8,358.6 L60.7,356.4 L61.5,354.1 L62.4,351.9 L63.2,349.6 L64.1,347.4 L64.9,345.1
 L65.8,342.9 L66.6,340.7 L67.5,338.4 L68.3,336.2 L69.2,333.9 L70.0,331.7 L70.9,329.5 L71.7,327.2
 L72.6,325.0 L73.4,322.8 L74.3,320.6 L75.1,318.3 L76.0,316.1 L76.8,313.9 L77.7,311.7 L78.5,309.5
 L79.4,307.3 L80.2,305.1 L81.1,302.9 L81.9,300.7 L82.8,298.5 L83.6,296.3 L84.5,294.1 L85.3,291.9
 L86.2,289.7 L87.0,287.5 L87.9,285.3 L88.7,283.1 L89.6,281.0 L90.4,278.8 L91.3,276.6 L92.1,274.5
 L93.0,272.3 L93.8,270.2 L94.7,268.0 L95.5,265.9 L96.4,263.7 L97.2,261.6 L98.1,259.5 L98.9,257.3
 L99.8,255.2 L100.6,253.1 L101.5,251.0 L102.3,248.9 L103.2,246.8 L104.0,244.7 L104.9,242.6 L105.7,240.5
 L106.5,238.4 L107.4,236.3 L108.2,234.2 L109.1,232.2 L109.9,230.1 L110.8,228.1 L111.6,226.0 L112.5,224.0
 L113.3,221.9 L114.2,219.9 L115.0,217.9 L115.9,215.9 L116.7,213.8 L117.6,211.8 L118.4,209.8 L119.3,207.8
 L120.1,205.8 L121.0,203.9 L121.8,201.9 L122.7,199.9 L123.5,198.0 L124.4,196.0 L125.2,194.1 L126.1,192.1
 L126.9,190.2 L127.8,188.3 L128.6,186.4 L129.5,184.4 L130.3,182.5 L131.2,180.6 L132.0,178.8 L132.9,176.9
 L133.7,175.0 L134.6,173.1 L135.4,171.3 L136.3,169.4 L137.1,167.6 L138.0,165.8 L138.8,163.9 L139.7,162.1
 L140.5,160.3 L141.4,158.5 L142.2,156.7 L143.1,155.0 L143.9,153.2 L144.8,151.4 L145.6,149.7 L146.5,147.9
 L147.3,146.2 L148.2,144.5 L149.0,142.7 L149.9,141.0 L150.7,139.3 L151.6,137.6 L152.4,136.0 L153.3,134.3
 L154.1,132.6 L155.0,131.0 L155.8,129.3 L156.6,127.7 L157.5,126.1 L158.3,124.5 L159.2,122.8 L160.0,121.3
 L160.9,119.7 L161.7,118.1 L162.6,116.5 L163.4,115.0 L164.3,113.4 L165.1,111.9 L166.0,110.4 L166.8,108.9
 L167.7,107.4 L168.5,105.9 L169.4,104.4 L170.2,102.9 L171.1,101.5 L171.9,100.0 L172.8,98.6 L173.6,97.1
 L174.5,95.7 L175.3,94.3 L176.2,92.9 L177.0,91.6 L177.9,90.2 L178.7,88.8 L179.6,87.5 L180.4,86.1
 L181.3,84.8 L182.1,83.5 L183.0,82.2 L183.8,80.9 L184.7,79.6 L185.5,78.3 L186.4,77.1 L187.2,75.8
 L188.1,74.6 L188.9,73.4 L189.8,72.2 L190.6,71.0 L191.5,69.8 L192.3,68.6 L193.2,67.5 L194.0,66.3
 L194.9,65.2 L195.7,64.1 L196.6,62.9 L197.4,61.8 L198.3,60.8 L199.1,59.7 L200.0,58.6 L200.8,57.6
 L201.7,56.5 L202.5,55.5 L203.4,54.5 L204.2,53.5 L205.1,52.5 L205.9,51.5 L206.8,50.6 L207.6,49.6
 L208.4,48.7 L209.3,47.8 L210.1,46.8 L211.0,46.0 L211.8,45.1 L212.7,44.2 L213.5,43.3 L214.4,42.5
 L215.2,41.7 L216.1,40.8 L216.9,40.0 L217.8,39.2 L218.6,38.5 L219.5,37.7 L220.3,36.9 L221.2,36.2
 L222.0,35.5 L222.9,34.8 L223.7,34.1 L224.6,33.4 L225.4,32.7 L226.3,32.0 L227.1,31.4 L228.0,30.8
 L228.8,30.2 L229.7,29.6 L230.5,29.0 L231.4,28.4 L232.2,27.8 L233.1,27.3 L233.9,26.7 L234.8,26.2
 L235.6,25.7 L236.5,25.2 L237.3,24.7 L238.2,24.3 L239.0,23.8 L239.9,23.4 L240.7,22.9 L241.6,22.5
 L242.4,22.1 L243.3,21.7 L244.1,21.4 L245.0,21.0 L245.8,20.7 L246.7,20.4 L247.5,20.0 L248.4,19.7
 L249.2,19.5 L250.1,19.2 L250.9,18.9 L251.8,18.7 L252.6,18.5 L253.5,18.2 L254.3,18.0 L255.2,17.8
 L256.0,17.7 L256.9,17.5 L257.7,17.4 L258.5,17.2 L259.4,17.1 L260.2,17.0 L261.1,16.9 L261.9,16.9
 L262.8,16.8 L263.6,16.8 L264.5,16.7 L265.3,16.7 L266.2,16.7 L267.0,16.7 L267.9,16.7 L268.7,16.8
 L269.6,16.8 L270.4,16.9 L271.3,17.0 L272.1,17.1 L273.0,17.2 L273.8,17.3 L274.7,17.4 L275.5,17.6
 L276.4,17.8 L277.2,17.9 L278.1,18.1 L278.9,18.3 L279.8,18.6 L280.6,18.8 L281.5,19.1 

Next, we create a grid which has a cell boundary exactly at the position of
the discontinuity of $g_2$.

In [8]:
var Nodes1 = new double[] {0, 2, Math.PI, 4.5, 2*Math.PI };         
var Grid1 = Grid1D.LineGrid(Nodes1);

We can get the total number of cells by using the following command:

In [9]:
Grid1.NumberOfCells

4

The recently created grid-object is not directly usable because it contains only the nodes of the grid. 

We have to create a **GridData**-object which provides all necessary transformation metrics, etc. .


In [10]:
var gdata1 = new GridData(Grid1);

### Projection onto the DG space
At this point, we are able to create the so-called ***DG fields*** to approximate ***$g_1$***
on ***grid1***. 

Therefore, we project ***$g_1$*** onto ***grid1*** using polynomial orders
of $p=2$ and $p=8$.


In [11]:
var g1_grid1_p2 = new SinglePhaseField(new Basis(gdata1, 2), "g1 with p2 at Grid 1");      
g1_grid1_p2.ProjectField(g1);

In [12]:
var g1_grid1_p8 = new SinglePhaseField(new Basis(gdata1, 8), "g1 with p8 at Grid 1");         
g1_grid1_p8.ProjectField(g1);

Now, let us plot the projected solution for $p=2$. 

By using the upsampling parameter, we can determine 
the amount of sampling points per cell.


In [13]:
var upsampling = 20;

In [14]:
var gp1 = new Gnuplot();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [15]:
gp1.PlotField(g1_grid1_p2,          
    new PlotFormat(lineColor: (LineColors)(1)),
    upsampling);

In [16]:
gp1.PlotNow() // shows the plot

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.2 
 
 
 
 
 -1 
 
 
 
 
 -0.8 
 
 
 
 
 -0.6 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 g1 with p2 at Grid 1 
 
 
 g1 with p2 at Grid 1 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M940.2,34.7 L982.4,34.7 M53.9,355.2 L68.1,313.1 L82.3,273.8 L96.5,237.4 L110.7,203.7 L125.0,172.9
 L139.2,144.9 L153.4,119.8 L167.6,97.4 L181.8,77.9 L196.0,61.2 L210.2,47.3 L224.4,36.2 L238.7,28.0
 L252.9,22.6 L267.1,20.0 L281.3,20.2 L295.5,23.2 L309.7,29.1 L323.9,37.7 L323.9,43.1 L332.0,53.4
 L340.2,64.3 L348.3,75.9 L356.4,88.1 L364.5,100.9 L372.6,114.3 L380.7,128.4 L388.8,143.1 L396.9,158.4
 L405.1,174.3 L413.2,190.8 L421.3,207.9 L429.4,225.7 L437.5,244.1 L445.6,263.1 L453.7,282.7 L461.8,303.0
 L469.9,323.8 L478.1,345.3 L478.1,336.3 L487.7,362.7 L497.4,388.0 L507.0,412.4 L516.7,435.7 L526.3,458.0
 L536.0,479.3 L545.6,499.6 L555.3,518.9 L564.9,537.2 L574.6,554.4 L584.2,570.7 L593.9,585.9 L603.5,600.2
 L613.2,613.4 L622.9,625.6 L632.5,636.8 L642.2,647.0 L651.8,656.2 L661.5,664.4 L661.5,667.3 L674.1,668.6
 L686.8,667.7 L699.5,664.8 L712.1,659.7 L724.8,652.5 L737.5,643.3 L750.2,631.9 L762.8,618.4 L775.5,602.8
 L788.2,585.2 L800.8,565.4 L813.5,543.5 L826.2,519.5 L838.9,493.4 L851.5,465.2 L864.2,434.8 L876.9,402.4
 L889.5,367.9 L902.2,331.3 '/>

### Computing the $L^2$-error
 Next, we learn how to compute the $L^2$-error for both approximations of $g_1$ with different polynomial degrees:


In [17]:
g1_grid1_p2.L2Error(g1)

0.00423225422106051

In [18]:
g1_grid1_p8.L2Error(g1)

4.039445326532082E-10

### Plotting the point-wise error

Now, we plot the point-wise error for the approximation of $g_1$ 
on **grid1** with a polynomial degree of 8.

In [36]:
int K = 20; // number of points per cell         
var gp2 = new Gnuplot();         
gp2.PlotLogError(g1_grid1_p8, g1, "g1 with p8 at Grid 1", 20,          
    new PlotFormat(lineColor: (LineColors)(1)));         
gp2.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1x10 -11 
 
 
 
 
 1x10 -10 
 
 
 
 
 1x10 -9 
 
 
 
 
 1x10 -8 
 
 
 
 
 1x10 -7 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 g1 with p8 at Grid 1 
 
 
 g1 with p8 at Grid 1 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M940.2,34.7 L982.4,34.7 M78.8,156.5 L92.6,240.4 L106.5,269.2 L120.3,274.3 L134.2,317.9 L148.0,259.7
 L161.8,339.3 L175.7,300.1 L189.5,266.4 L203.3,335.7 L217.2,314.1 L231.0,267.5 L244.9,320.3 L258.7,320.3
 L272.5,265.3 L286.4,348.3 L300.2,275.4 L314.0,286.7 L327.9,248.6 L341.7,169.3 L341.7,520.3 L349.6,601.6
 L357.5,635.6 L365.4,631.3 L373.3,691.3 L381.2,619.4 L389.1,683.5 L397.0,668.2 L404.9,623.5 L412.8,678.1
 L420.7,683.2 L428.6,623.2 L436.5,663.5 L444.4,688.1 L452.3,618.1 L460.2,684.2 L468.1,631.1 L476.0,631.6
 L483.9,599.7 L491.8,517.3 L491.8,401.4 L501.2,484.0 L510.6,515.4 L520.0,515.8 L529.4,567.4 L538.8,502.6
 L548.2,573.9 L557.6,547.1 L567.0,507.9 L576.4,569.2 L585.8,561.6 L595.2,508.3 L604.6,554.2 L614.0,567.2
 L623.4,504.5 L632.8,578.0 L642.2,516.2 L651.6,521.5 L661.0,486.9 L670.4,405.9 L670.4,235.4 L682.7,315.5
 L695.0,352.0 L707.4,343.4 L719.7,411.1 L732.0,332.6 L744.4,391.0 L756.7,385.1 L769.1,335.5 L781.4,385.0
 L793.7,400.3 L806.1,334.7 L818.4,370.8 L830.7,404.5 L843.1,328.6 L855.4,389.7 L867.8,342.6 L880.1,339.6
 L892.4,309.6 L904.8,226.3 '/>

### Decay behavior of the DG modes for smooth and non-smooth functions
We investigate the decay behavior of the DG modes for smooth and non-smooth 
functions. 

For this purpose, we create a second grid which has the
discontinuity of $g_2$ within a cell and project $g_2$ onto this grid
like mentioned above.


In [20]:
var Nodes2      = new double[] {0, 2, 4.5, 2*Math.PI };         
var Grid2       = Grid1D.LineGrid(Nodes2);         
var gdata2      = new GridData(Grid2);         
var g2_grid2_p8 = new SinglePhaseField(new Basis(gdata2, 8), "g2_p8 at Grid2");         
g2_grid2_p8.ProjectField(g2);

The cell coordinates can be extracted by using the **Coordinates** parameter.

In [21]:
double[] cell1 = g2_grid2_p8.Coordinates.GetRow(1);          
    // coord. in cell 1 (with kink)

In [22]:
double[] cell0 = g2_grid2_p8.Coordinates.GetRow(0);          
    // coord. in cell 0 (smooth)

In [23]:
double[] cell2 = g2_grid2_p8.Coordinates.GetRow(2);          
    // coord. in cell 2 (smooth)

Only the absolute value shall be plotted. We use a for-loop to replace the data in 
**cell0**, **cell1** and **cell2** by their absolute values.


In [24]:
for(int i = 0; i < cell0.Length; i++) {         
    cell0[i] = Math.Abs(cell0[i]);         
    cell1[i] = Math.Abs(cell1[i]);         
    cell2[i] = Math.Abs(cell2[i]);         
 }

In [37]:
Plot(X1:null, Y1:cell1, Name1:"disc. cell", Format1:"*-magenta",
     X2:null, Y2:cell0, Name2:"cell0",      Format2:"o-blue",
     X3:null, Y3:cell2, Name3:"cell2",      Format3:"o-red")

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 disc. cell 
 
 
 disc. cell 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cell0 
 
 
 cell0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cell2 
 
 
 cell2

#### Note
Using a shortcut for the for-loop above, the absolute values in **cell0**
can also be stored using the following command:

In [26]:
double[] cell0 = g2_grid2_p8.Coordinates.GetRow(0)         
    .Select(d => Math.Abs(d)).ToArray();

Now, we would like to plot the logarithm (use **Math.Log10(...)**) of the absolute
values of the DG coordinates.

In [38]:
Plot(X1:null, Y1:cell1, Name1:"disc. cell", Format1:"*-magenta",
     X2:null, Y2:cell0, Name2:"cell0",      Format2:"o-blue",
     X3:null, Y3:cell2, Name3:"cell2",      Format3:"o-red",
     logY:true)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00000001 
 
 
 
 
 0.00000010 
 
 
 
 
 0.00000100 
 
 
 
 
 0.00001000 
 
 
 
 
 0.00010000 
 
 
 
 
 0.00100000 
 
 
 
 
 0.01000000 
 
 
 
 
 0.10000000 
 
 
 
 
 1.00000000 
 
 
 
 
 10.00000000 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 disc. cell 
 
 
 disc. cell 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cell0 
 
 
 cell0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cell2 
 
 
 cell2

### Convergence study
In this section, we learn how to perform a convergence study for $g_2$
for two different sequences of grid resolutions and different polynomial
orders. 

Therefore, we define two different sequences of grid resolutions:


In [28]:
int[][] ResSeq = new int[2][];

Grid resolutions so that the kink in **g2** is located at a cell boundary:

In [29]:
ResSeq[0] = new int[] { 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048 };

Grid resolutions so that the kink in **g2** is located within a cell:

In [30]:

ResSeq[1] = new int[] { 3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047 };

We save our errors into a multidimensional array by looping over 

- the resolution sequence
- the polynomial order
- the resolution

In [31]:
var Errors = MultidimensionalArray.Create(2, 5, ResSeq[0].Length);

In [32]:
for(int i = 0; i < 2; i++) { // loop over the resolution sequence         
    for(int p = 0; p <= 4; p++) { // loop over polynomial orders         
        for(int k = 0; k < ResSeq[i].Length; k++) { // loop over different resolutions        
 
            Console.Write("polynomial order {1}"+         
            ",\tResolution {0}... ", ResSeq[i][k], p);         
 
            var grid  = Grid1D.LineGrid(GenericBlas.Linspace(0,         
            2.0*Math.PI, ResSeq[i][k] + 1));           
                 // number of nodes == number of cells + 1         
 
            var gData = new GridData(grid);         
 
            var g2_h  = new SinglePhaseField(new Basis(gData, p));         
 
            g2_h.ProjectField(g2);         
 
            Errors[i,p,k] = g2_h.L2Error(g2);         
 
            Console.WriteLine("\tdone: L2 error is {0:0.###e-00}.", Errors[i,p,k]);         
        }         
    }         
 }

polynomial order 0,	Resolution 4... 	done: L2 error is 1.791e-01.
polynomial order 0,	Resolution 8... 	done: L2 error is 4.536e-02.
polynomial order 0,	Resolution 16... 	done: L2 error is 1.138e-02.
polynomial order 0,	Resolution 32... 	done: L2 error is 2.846e-03.
polynomial order 0,	Resolution 64... 	done: L2 error is 7.118e-04.
polynomial order 0,	Resolution 128... 	done: L2 error is 1.779e-04.
polynomial order 0,	Resolution 256... 	done: L2 error is 4.449e-05.
polynomial order 0,	Resolution 512... 	done: L2 error is 1.112e-05.
polynomial order 0,	Resolution 1024... 	done: L2 error is 2.781e-06.
polynomial order 0,	Resolution 2048... 	done: L2 error is 6.951e-07.
polynomial order 1,	Resolution 4... 	done: L2 error is 2.155e-02.
polynomial order 1,	Resolution 8... 	done: L2 error is 2.739e-03.
polynomial order 1,	Resolution 16... 	done: L2 error is 3.438e-04.
polynomial order 1,	Resolution 32... 	done: L2 error is 4.302e-05.
polynomial order 1,	Resolution 64... 	done: L2 error is 5.3

In [33]:
/// NUnit test (few random tests)
Assert.LessOrEqual(Errors[1,4,9],8E-06);       
Assert.LessOrEqual(Errors[1,3,9],7.5E-06);       
Assert.LessOrEqual(Errors[1,2,9],2E-05);       
Assert.LessOrEqual(Errors[1,1,9],2E-05);       
Assert.LessOrEqual(Errors[0,3,9],1E-12);       
Assert.LessOrEqual(Errors[0,3,9],1E-12);       
Assert.LessOrEqual(Errors[1,4,0],0.25);       
Assert.LessOrEqual(Errors[0,0,0],0.2);       
Assert.LessOrEqual(Errors[0,3,0],1E-03);

We plot the error for the grids which have the kink at the cell boundary,
there we reach spectral convergence:

In [34]:
var hValues = ResSeq[0].Select(J => Math.PI*2.0/J);
Plot(X1:hValues, Y1:Errors.ExtractSubArrayShallow(0,0,-1).To1DArray(),
     Name1:"grid1,p0", Format1:"o-red",
     X2:hValues, Y2:Errors.ExtractSubArrayShallow(0,1,-1).To1DArray(),
     Name2:"grid1,p1", Format2:"o-blue",
     X3:hValues, Y3:Errors.ExtractSubArrayShallow(0,2,-1).To1DArray(),
     Name3:"grid1,p2", Format3:"o-green",
     X4:hValues, Y4:Errors.ExtractSubArrayShallow(0,3,-1).To1DArray(),
     Name4:"grid1,p3", Format4:"o-magenta",
     X5:hValues, Y5:Errors.ExtractSubArrayShallow(0,4,-1).To1DArray(),
     Name5:"grid1,p4", Format5:"o-orange",
     logX:true, logY:true)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1x10 -16 
 
 
 
 
 1x10 -14 
 
 
 
 
 1x10 -12 
 
 
 
 
 1x10 -10 
 
 
 
 
 1x10 -8 
 
 
 
 
 1x10 -6 
 
 
 
 
 0.0001 
 
 
 
 
 0.01 
 
 
 
 
 1 
 
 
 
 
 0.001 
 
 
 
 
 0.01 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 grid1,p0 
 
 
 grid1,p0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p1 
 
 
 grid1,p1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p2 
 
 
 grid1,p2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p3 
 
 
 grid1,p3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p4 
 
 
 grid1,p4

Now we plot the error for the grids which have the kink within a cell;
due to the low regularity, the convergence of the DG method
degenerates:

In [35]:
var hValues = ResSeq[0].Select(J => Math.PI*2.0/J);
Plot(X1:hValues, Y1:Errors.ExtractSubArrayShallow(1,0,-1).To1DArray(),
     Name1:"grid1,p0", Format1:"o-red",
     X2:hValues, Y2:Errors.ExtractSubArrayShallow(1,1,-1).To1DArray(),
     Name2:"grid1,p1", Format2:"o-blue",
     X3:hValues, Y3:Errors.ExtractSubArrayShallow(1,2,-1).To1DArray(),
     Name3:"grid1,p2", Format3:"o-green",
     X4:hValues, Y4:Errors.ExtractSubArrayShallow(1,3,-1).To1DArray(),
     Name4:"grid1,p3", Format4:"o-magenta",
     X5:hValues, Y5:Errors.ExtractSubArrayShallow(1,4,-1).To1DArray(),
     Name5:"grid1,p4", Format5:"o-orange",
     logX:true, logY:true)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000010 
 
 
 
 
 0.0000100 
 
 
 
 
 0.0001000 
 
 
 
 
 0.0010000 
 
 
 
 
 0.0100000 
 
 
 
 
 0.1000000 
 
 
 
 
 1.0000000 
 
 
 
 
 0.001 
 
 
 
 
 0.01 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 grid1,p0 
 
 
 grid1,p0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p1 
 
 
 grid1,p1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p2 
 
 
 grid1,p2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p3 
 
 
 grid1,p3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 grid1,p4 
 
 
 grid1,p4

## Where to go from here? (Further Reading)
This tutorial addressed the very basics of setting up a **BoSSS~application**, namely grid instantiation, the $L^2$-projection of functions onto the DG space and performing a spatial convergence study. Where do you go from here? We recommend that you continue with other relevant basics as provided in the tutorials dealing with the creation of a spatial operator, explicit time integration and the implementation of numerical fluxes.